In [ ]:
import pandas as pd
import numpy as np
import joblib

In [ ]:
user_input = {
    'inning': 1,
    'cum_runs': 80,
    'cum_wickets': 5,
    'overs_completed': 12.0,
    'target': 0,
    'batting_team': "Mumbai Indians",
    'bowling_team': "Chennai Super Kings",
    'venue': "Wankhede Stadium"
}

In [ ]:
# 1. Compute Derived Features
# ---------------------------
# Compute current run rate
if user_input['overs_completed'] > 0:
    current_run_rate = user_input['cum_runs'] / user_input['overs_completed']
else:
    current_run_rate = 0

# Compute required run rate
remaining_overs = 20 - user_input['overs_completed']
if user_input['inning'] == 2 and remaining_overs > 0:
    required_run_rate = (user_input['target'] - user_input['cum_runs']) / remaining_overs
else:
    required_run_rate = 0

In [ ]:
# ---------------------------
# 2. Load Saved Encoders and Model
# ---------------------------
# Load the team and venue LabelEncoders
le_team = joblib.load('le_team.pkl')
le_venue = joblib.load('le_venue.pkl')


model = joblib.load('final_rf_model.pkl')

In [ ]:
# ---------------------------
# 3. Encode Categorical Variables
# ---------------------------
# Encode batting team and bowling team using the team encoder
batting_team_encoded = le_team.transform([user_input['batting_team']])[0]
bowling_team_encoded = le_team.transform([user_input['bowling_team']])[0]

# Encode the venue using the venue encoder
venue_canonical_encoded = le_venue.transform([user_input['venue']])[0]

In [ ]:
# ---------------------------
# 4. Create Input DataFrame for Inference
# ---------------------------

input_data = {
    'inning': [user_input['inning']],
    'cum_runs': [user_input['cum_runs']],
    'cum_wickets': [user_input['cum_wickets']],
    'current_run_rate': [current_run_rate],
    'required_run_rate': [required_run_rate],
    'target': [user_input['target']],
    'batting_team_encoded': [batting_team_encoded],
    'bowling_team_encoded': [bowling_team_encoded],
    'venue_canonical_encoded': [venue_canonical_encoded]
}

input_df = pd.DataFrame(input_data)

In [ ]:

# The model predicts a binary outcome: 1 indicates the batting team wins, 0 indicates the bowling team wins.
prediction = model.predict(input_df)[0]
predicted_probabilities = model.predict_proba(input_df)[0]

# ---------------------------
# 6. Map Prediction to Team Names
# ---------------------------
if prediction == 1:
    predicted_winner = user_input['batting_team']
else:
    predicted_winner = user_input['bowling_team']

print("Predicted Winner:", predicted_winner)
print("Prediction Probabilities (Loss, Win):", predicted_probabilities)

Predicted Winner: Mumbai Indians
Prediction Probabilities (Loss, Win): [0.18 0.82]
